## Problem Analysis

## Data Collection

### Importing libraries

In [ ]:
# Import all necessary libraries
import pandas as pd
import ydata_profiling
import os
import numpy as np
import gc
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import mlflow

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

### Collecting data

In [ ]:
# Paso 1: Ingesta y unificación de datos con polars

# Rutas de los archivos
setA_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_setA.parquet"
setB_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_setB.parquet"
unified_path = r"C:\repos\physionet-sepsis-forecasting\data\raw\all_patients_unified.parquet"

In [ ]:
# Leer ambos datasets con pandas
df_a = pd.read_parquet(setA_path)
df_b = pd.read_parquet(setB_path)

In [ ]:
# Mostrar las columnas que NO tienen en común
print("Columnas en Set A pero no en Set B:", set(df_a.columns) - set(df_b.columns))
print("Columnas en Set B pero no en Set A:", set(df_b.columns) - set(df_a.columns))

In [ ]:
# Asegurar que ambos tengan las mismas columnas eliminando las que no coinciden
common_cols = list(set(df_a.columns) & set(df_b.columns))
df_a = df_a[common_cols]
df_b = df_b[common_cols]

In [ ]:
# Unificar
df = pd.concat([df_a, df_b])

# Guardar el dataset unificado
df.to_parquet(unified_path)
print(f"Dataset unificado guardado en {unified_path}")

In [ ]:
# Limpiar memoria eliminando df_a y df_b
del df_a
del df_b
gc.collect()

## EDA

In [ ]:
# Paso 2: Análisis exploratorio
df = pd.read_parquet(unified_path)
df.shape

In [ ]:
# Ver las primeras filas
df.head()

In [ ]:
# Descripción estadística rápida
df.describe()

In [ ]:
# Conteo de valores nulos por columna con porcentaje
null_counts = df.isnull().sum()
null_percent = (null_counts / len(df)) * 100
null_df = pd.DataFrame({'null_count': null_counts, 'null_percent': null_percent})
null_df = null_df[null_df['null_count'] >= 0].sort_values(by='null_percent', ascending=False)
print(null_df)

In [ ]:
# Analizar desbalance de clases en la variable objetivo SepsisLabel
sepsis_counts = df['SepsisLabel'].value_counts()
sepsis_percent = (sepsis_counts / len(df)) * 100
sepsis_classes = pd.DataFrame({'sepsis_counts': sepsis_counts, 'sepsis_percent': sepsis_percent})
sepsis_classes = sepsis_classes[sepsis_classes['sepsis_counts'] >= 0].sort_values(by='sepsis_percent', ascending=False)

print(sepsis_classes)

In [ ]:
# Generar un reporte con ydata-profiling

profile = ydata_profiling.ProfileReport(df, title="Reporte de Análisis Exploratorio", explorative=True, minimal=True)
# Si el directorio no existe, crearlo
os.makedirs(os.path.dirname(r"C:\repos\physionet-sepsis-forecasting\data\reports"), exist_ok=True)
# Guardar el reporte
profile_path = r"C:\repos\physionet-sepsis-forecasting\data\reports\eda_report.html"
profile.to_file(profile_path)

## Data Cleaning & Preprocessing

### Imputation

In [ ]:
# Imputar valores nulos con la media usando pandas y para las variables categóricas con la moda
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    mean_value = df[col].mean()
    df[col] = df[col].fillna(mean_value)

categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_cols:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)

### Data Scalation

In [ ]:
# Aplicar StandardScaler de sklearn a las columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Excluir columnas que no deben ser escaladas
exclude_cols = ['SepsisLabel', 'patient_id', 'ICULOS']
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

## Modeling a LSTM

Transformamos el dataframe (que tiene una fila por hora por paciente) en un formato (número_de_muestras, longitud_de_secuencia, número_de_características), que es lo que un LSTM espera.

In [ ]:
# Ordenamos y agrupamos por paciente
df = df.sort_values(by=['patient_id', 'ICULOS'])

### Define features and target

In [ ]:
# Denifimos características y etiquetas
features_cols = [col for col in df.columns if col not in ['SepsisLabel', 'patient_id']]
target_col = 'SepsisLabel'

### Sliding windows

In [ ]:
# Calcular la secuencia minima y máxima por paciente
seq_lengths = df.groupby('patient_id').size()
min_seq_length = seq_lengths.min()
max_seq_length = seq_lengths.max() 
print(f"Longitud mínima de secuencia por paciente: {min_seq_length}")
print(f"Longitud máxima de secuencia por paciente: {max_seq_length}")

In [ ]:
# Contar cuantos pacientes hay con la secuencia mínima y máxima
min_seq_count = (seq_lengths == min_seq_length).sum()
max_seq_count = (seq_lengths == max_seq_length).sum()
print(f"Número de pacientes con la secuencia mínima ({min_seq_length}): {min_seq_count}")
print(f"Número de pacientes con la secuencia máxima ({max_seq_length}): {max_seq_count}")

In [ ]:
# Creamos secuencias para LSTM (Ventanas deslizantes)
# Parámetros
sequence_length = min_seq_length # Usar la secuencia minima de datos como para predecir la siguiente
X_sequences = []
y_sequences = []

# Agrupar por paciente para no mezclar datos de diferentes personas
grouped = df.groupby('patient_id')

for _, group in grouped:
    features = group[features_cols].values
    target = group[target_col].values
    
    # Crear ventanas deslizantes para cada paciente
    for i in range(len(group) - sequence_length):
        X_sequences.append(features[i:i + sequence_length])
        y_sequences.append(target[i + sequence_length])

# Convertir a arrays de NumPy
X = np.array(X_sequences)
y = np.array(y_sequences)

print(f"Forma de las secuencias de entrada (X): {X.shape}")
print(f"Forma de las etiquetas de salida (y): {y.shape}")

# La salida de X.shape debería ser (num_muestras, min_seq_length, num_features)

### Split in Train & Test

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train shapes: {X_train.shape}, {y_train.shape}")
print(f"Validation shapes: {X_val.shape}, {y_val.shape}")
print(f"Test shapes: {X_test.shape}, {y_test.shape}")

### Save processed data

Ahora que tenemos los datos en el formato correcto los guardaremos en la carpeta `data\processed`. No guardaremos un .parquet porque ya no es un dataframe 2D. Usaremos el formato de NumPy (.npy) que es ideal para arrays multidimensionales.

In [ ]:
processed_dir = r"C:\repos\physionet-sepsis-forecasting\data\processed"
os.makedirs(processed_dir, exist_ok=True)

np.save(os.path.join(processed_dir, 'X_train.npy'), X_train)
np.save(os.path.join(processed_dir, 'y_train.npy'), y_train)
np.save(os.path.join(processed_dir, 'X_val.npy'), X_val)
np.save(os.path.join(processed_dir, 'y_val.npy'), y_val)
np.save(os.path.join(processed_dir, 'X_test.npy'), X_test)
np.save(os.path.join(processed_dir, 'y_test.npy'), y_test)

print("Datos procesados y divididos guardados en data/processed/")

### Versionamiento con DVC:
Ahora, añade estos archivos .npy a DVC, tal como hiciste con el archivo Parquet, y sube los cambios.

In [ ]:
dvc add data/processed
git add data/processed.dvc
git commit -m "feat: Create time-series sequences for LSTM model"
dvc push

### Fase 2: Implementación y Entrenamiento del Modelo LSTM

**Objetivo**: Construir la arquitectura del modelo, entrenarlo con los datos secuenciales y registrar los resultados con MLflow.

2.1. Adaptar el Notebook para el Entrenamiento (o crear scripts/train.py):
Te recomiendo encarecidamente mover el código de entrenamiento del notebook a un script scripts/train.py para seguir las buenas prácticas del repositorio.
2.2. Crear los DataLoaders de PyTorch:
PyTorch usa DataLoader para gestionar los datos en batches de manera eficiente.

In [ ]:
# Liberar memoria de las variables originales
del X_train, y_train, X_val, y_val, X_test, y_test, df, X_temp, y_temp
gc.collect()

In [ ]:
# Cargar los datos desde data/processed
processed_dir = r"C:\repos\physionet-sepsis-forecasting\data\processed"
X_train = np.load(os.path.join(processed_dir, 'X_train.npy'))
y_train = np.load(os.path.join(processed_dir, 'y_train.npy'))
X_val = np.load(os.path.join(processed_dir, 'X_val.npy'))
y_val = np.load(os.path.join(processed_dir, 'y_val.npy'))

# Convertir a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

# Crear TensorDatasets y DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)


num_workers = os.cpu_count()  # Usa todos los núcleos disponibles

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=num_workers)

#train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

### 2.3. Definir la Arquitectura del Modelo LSTM:
Aquí definimos las capas del modelo.

In [ ]:
class SepsisLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(SepsisLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Inicializar estados ocultos
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Pasar por el LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Tomar la salida del último paso de tiempo y pasarla por la capa densa
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

### 2.4. Bucle de Entrenamiento y Evaluación:
Este es el corazón del entrenamiento, donde iteramos sobre los datos, calculamos la pérdida y ajustamos los pesos del modelo.

In [ ]:
# Tomar solo el 5% del dataset de entrenamiento para pruebas rápidas
sample_frac = 0.05
num_samples = int(X_train_tensor.shape[0] * sample_frac)

X_train_small = X_train_tensor[:num_samples]
y_train_small = y_train_tensor[:num_samples]

train_dataset_small = TensorDataset(X_train_small, y_train_small)
train_loader_small = DataLoader(train_dataset_small, batch_size=64, shuffle=True, num_workers=os.cpu_count())

# Configuración del modelo y entrenamiento rápido
input_size = X_train_small.shape[2] # Número de features
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 10
learning_rate = 0.001

model = SepsisLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss() # Binary Cross-Entropy para clasificación binaria
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# --- INTEGRACIÓN CON MLFLOW ---
mlflow.set_tracking_uri("http://100.24.7.21:5000")
with mlflow.start_run(nested=True) as run:
    mlflow.log_params({"hidden_size": hidden_size, "num_layers": num_layers, "epochs": num_epochs, "train_frac": sample_frac})

    for epoch in range(num_epochs):
        model.train()
        for batch_X, batch_y in train_loader_small:
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Bucle de validación (calcular métricas como AUC-ROC en el val_loader)
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                outputs = model(batch_X)
                all_preds.extend(outputs.cpu().numpy())
                all_labels.extend(batch_y.cpu().numpy())
        
        auc = roc_auc_score(all_labels, all_preds)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val AUC: {auc:.4f}")
        mlflow.log_metric("val_auc", auc, step=epoch)

        mlflow.pytorch.log_model(model, "lst_model_8_sw_test")

In [ ]:
# Configuración del modelo y entrenamiento
input_size = X_train_tensor.shape[2] # Número de features
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 10
learning_rate = 0.001

model = SepsisLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss() # Binary Cross-Entropy para clasificación binaria
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# --- INTEGRACIÓN CON MLFLOW ---
mlflow.set_tracking_uri("http://100.24.7.21:5000")
with mlflow.start_run() as run:
    mlflow.log_params({"hidden_size": hidden_size, "num_layers": num_layers, "epochs": num_epochs})

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Bucle de validación (calcular métricas como AUC-ROC en el val_loader)
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    auc = roc_auc_score(all_labels, all_preds)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val AUC: {auc:.4f}")
    mlflow.log_metric("val_auc", auc, step=epoch)

    mlflow.pytorch.log_model(model, "lst_model_8_sw")

In [ ]:
# Configuración del modelo y entrenamiento
input_size = X_train.shape[2] # Número de features
hidden_size = 128
num_layers = 2
output_size = 1
num_epochs = 10
learning_rate = 0.001

model = SepsisLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss() # Binary Cross-Entropy para clasificación binaria
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# --- INTEGRACIÓN CON MLFLOW ---
mlflow.set_tracking_uri("http://54.226.120.46:5000")
with mlflow.start_run() as run:
    mlflow.log_params({"hidden_size": hidden_size, "num_layers": num_layers, "epochs": num_epochs})

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Bucle de validación (calcular métricas como AUC-ROC en el val_loader)
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    auc = roc_auc_score(all_labels, all_preds)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val AUC: {auc:.4f}")
    mlflow.log_metric("val_auc", auc, step=epoch)

    mlflow.pytorch.log_model(model, "lst_model_8_sw")